This notebook is a step by step guide about how to train a deep neural network (DNN) in the DeepDeconv framework.

In [1]:
## Set up the sys.path in order to be able to import our modules
import os
import sys
module_path = os.path.abspath(os.path.join('../python'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('/data/alpha-transform'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.environ["CUDA_VISIBLE_DEVICES"]="1" #"0" for the 1st GPU or "1" to use the 2nd

In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

###################################
# TensorFlow wizardry
config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.75 

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Using TensorFlow backend.


We load the class of the network. This class must inherit from the DeepNet superclass. The method build_model has to be redefined in the child class with the wanted architecture. In our work, we use the network defined in deconvnNet. 

In [3]:
from DeepDeconv.deepnetFCS.DeconvNet_custom import DeconvNet

/home/cben-ali/Programs/deep-deconv/python/shape_constraint/cadmos_lib.py:149: ComplexWarning: Casting complex values to real discards the imaginary part
  coeff[s] = trafo.adjoint_transform(temp, do_norm=False)


# 1) Alexis net, preshape
Training of Alexis' network with shape constraint and gamma=0. This will be used to check if adding the shaping constraint but keeping gamma=0 gives the same results as Alexis' metwork. It will also be used as a base for training the restarts.

In [4]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.01
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_Target_window_iter1'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'
    
dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)

Instructions for updating:
Colocations handled automatically by placer.


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeconvNet_custom.py:295: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=[<tf.Tenso...)`
  self.model = Model(input = [inputs,window,norm], outputs =output)


Tensor("loss/conv2d_8_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss/conv2d_8_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 96, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 96, 32)   128         conv2d_1[0][0]                   
___________________________________________________

activation_14 (Activation)      (None, 24, 24, 24)   0           batch_normalization_14[0][0]     
__________________________________________________________________________________________________
separable_conv2d_12 (SeparableC (None, 24, 24, 12)   504         activation_14[0][0]              
__________________________________________________________________________________________________
concatenate_11 (Concatenate)    (None, 24, 24, 36)   0           concatenate_10[0][0]             
                                                                 separable_conv2d_12[0][0]        
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 24, 24, 36)   144         concatenate_11[0][0]             
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 24, 24, 36)   0           batch_normalization_15[0][0]     
__________

In [ ]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'new_targets/'+'image-*-multihdu_newtar.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(data_directory+'Target_Window/'+'Target*')
win_files.sort()
print(win_files)

SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

['/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-00-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-01-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-02-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-03-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-04-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-05-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-06-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-07-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-08-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-09-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_eu

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.01_Target_window_iter1.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Instructions for updating:
Use tf.cast instead.


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:399: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=([array([[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


Epoch 1/20
5947/5947 [==============================] - 5765s 969ms/step - loss: 0.5385 - weighted_mean_squared_error: 0.3615 - weighted_shape_metric: 0.1770 - val_loss: 0.2117 - val_weighted_mean_squared_error: 0.0454 - val_weighted_shape_metric: 0.1663

Epoch 00001: val_loss improved from inf to 0.21170, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.01_Target_window_iter1.hdf5
2019-08-27_15:06:31 - Epoch: 1, val_loss: 0.211698, val_weighted_mean_squared_error: 0.045359, val_weighted_shape_metric: 0.166339, loss: 0.538807, weighted_mean_squared_error: 0.361877, weighted_shape_metric: 0.176929
Epoch 2/20
5947/5947 [==============================] - 5694s 958ms/step - loss: 0.0725 - weighted_mean_squared_error: 0.0346 - weighted_shape_metric: 0.0379 - val_loss: 0.2242 - val_weighted_mean_squared_error: 0.1326 - val_weighted_shape_metric: 0.0915

Epoch 00002: val_loss did not improve from 0.21170
2019-08-27_16:41:25 - Epoch: 2, val_loss: 0.224155,


Epoch 00015: val_loss did not improve from 0.08137
2019-08-28_13:14:17 - Epoch: 15, val_loss: 0.105229, val_weighted_mean_squared_error: 0.021994, val_weighted_shape_metric: 0.083235, loss: 0.043635, weighted_mean_squared_error: 0.026432, weighted_shape_metric: 0.017203
Epoch 16/20
5947/5947 [==============================] - 5691s 957ms/step - loss: 0.0343 - weighted_mean_squared_error: 0.0219 - weighted_shape_metric: 0.0124 - val_loss: 0.0898 - val_weighted_mean_squared_error: 0.0214 - val_weighted_shape_metric: 0.0684

Epoch 00016: val_loss did not improve from 0.08137
2019-08-28_14:49:08 - Epoch: 16, val_loss: 0.089806, val_weighted_mean_squared_error: 0.021409, val_weighted_shape_metric: 0.068397, loss: 0.034289, weighted_mean_squared_error: 0.021866, weighted_shape_metric: 0.012422
Epoch 17/20
5947/5947 [==============================] - 5692s 957ms/step - loss: 0.0346 - weighted_mean_squared_error: 0.0218 - weighted_shape_metric: 0.0128 - val_loss: 0.0956 - val_weighted_mean_sq

In [ ]:
dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.001_Gaussian_window.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1,keep_best_loss=True,initial_epoch=13)#,keep_best_loss=False by default

In [7]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=500
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)

loss (None, 96, 96)
temp (None, 1, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
loss (None, 96, 96)
temp (None, 1, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 96, 96, 32)   288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_42 (BatchNo (None, 96, 96, 32)   128         conv2d_9[0][0]                   
__________________________________________________________________________________________________
activation_42 (Activation)      (None, 96, 96, 32)   0           batch_normalizatio

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma500_shearlet.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/20
5947/5947 [==============================] - 5865s 986ms/step - loss: 1.3739 - weighted_mean_squared_error: 0.3273 - weighted_shearlet_metric: 1.0465 - val_loss: 0.0438 - val_weighted_mean_squared_error: 0.0346 - val_weighted_shearlet_metric: 0.0092

Epoch 00001: val_loss improved from inf to 0.04383, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma500_shearlet.hdf5
2019-07-28_03:39:35 - Epoch: 1, val_loss: 0.043833, val_weighted_mean_squared_error: 0.034587, val_weighted_shearlet_metric: 0.009246, loss: 1.375721, weighted_mean_squared_error: 0.327732, weighted_shearlet_metric: 1.047990
Epoch 2/20
5947/5947 [==============================] - 5890s 990ms/step - loss: 0.0420 - weig

5947/5947 [==============================] - 5893s 991ms/step - loss: 0.0213 - weighted_mean_squared_error: 0.0208 - weighted_shearlet_metric: 5.3937e-04 - val_loss: 0.2116 - val_weighted_mean_squared_error: 0.0952 - val_weighted_shearlet_metric: 0.1164

Epoch 00015: val_loss did not improve from 0.02138
2019-07-29_02:34:23 - Epoch: 15, val_loss: 0.211591, val_weighted_mean_squared_error: 0.095193, val_weighted_shearlet_metric: 0.116397, loss: 0.021308, weighted_mean_squared_error: 0.020768, weighted_shearlet_metric: 0.000539
Epoch 16/20
5947/5947 [==============================] - 5892s 991ms/step - loss: 0.0215 - weighted_mean_squared_error: 0.0209 - weighted_shearlet_metric: 5.6169e-04 - val_loss: 0.0279 - val_weighted_mean_squared_error: 0.0236 - val_weighted_shearlet_metric: 0.0043

Epoch 00016: val_loss did not improve from 0.02138
2019-07-29_04:12:35 - Epoch: 16, val_loss: 0.027895, val_weighted_mean_squared_error: 0.023610, val_weighted_shearlet_metric: 0.004285, loss: 0.021446

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.05
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

In [4]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)

Instructions for updating:
Colocations handled automatically by placer.


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeconvNet_custom.py:298: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`
  self.model = Model(input = inputs, outputs = output)


loss (None, 96, 96)
temp (None, 1, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
loss (None, 96, 96)
temp (None, 1, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 96, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 96, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 96, 32)   0           batch_normalizatio

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0_shearlet.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Instructions for updating:
Use tf.cast instead.


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:399: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=(array([[[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


Epoch 1/20
5947/5947 [==============================] - 5852s 984ms/step - loss: 0.3077 - weighted_mean_squared_error: 0.3077 - weighted_shearlet_metric: 0.0000e+00 - val_loss: 0.0288 - val_weighted_mean_squared_error: 0.0288 - val_weighted_shearlet_metric: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.02878, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0_shearlet.hdf5
2019-07-31_10:01:05 - Epoch: 1, val_loss: 0.028780, val_weighted_mean_squared_error: 0.028780, val_weighted_shearlet_metric: 0.000000, loss: 0.308083, weighted_mean_squared_error: 0.308083, weighted_shearlet_metric: 0.000000
Epoch 2/20
5947/5947 [==============================] - 5976s 1s/step - loss: 0.0302 - weighted_mean_squared_error: 0.0302 - weighted_shearlet_metric: 0.0000e+00 - val_loss: 0.0313 - val_weighted_mean_squared_error: 0.0313 - val_weighted_shearlet_metric: 0.0000e+00

Epoch 00002: val_loss did not improve from 0.02878
2019-07-31_11:40:41 - Epoch: 2, va

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.1
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

#STOP

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles.hdf5",
             shape_constraint=True,gamma=0) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

Loading model...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles.hdf5
Renaming as...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles
Tensor("loss_3/conv2d_24_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window_3:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm_3:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss_3/conv2d_24_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
[<tf.Tensor 'input_3_1:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'window_3:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'norm_3:0' shape=(?, 6, 1, 1) dtype=float32>]
['input_3', 'window', 'norm']


In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_test_ref_restart_newtar_gfiles.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1,keep_best_loss=True,initial_epoch=10)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_test_ref_restart_newtar_gfiles.hdf5
Memory usage for the model + one batch (GB): 2.809000
Current best_parameters: 4 0.02279198535978794
Epoch 11/20


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:363: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=([array([[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=10, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


5947/5947 [==============================] - 5738s 965ms/step - loss: 0.0236 - weighted_mean_squared_error: 0.0236 - weighted_shape_metric: 0.0000e+00 - val_loss: 0.0479 - val_weighted_mean_squared_error: 0.0479 - val_weighted_shape_metric: 0.0000e+00

Epoch 00011: val_loss did not improve from 0.02279
2019-07-11_09:23:42 - Epoch: 11, val_loss: 0.047860, val_weighted_mean_squared_error: 0.047860, val_weighted_shape_metric: 0.000000, loss: 0.023645, weighted_mean_squared_error: 0.023645, weighted_shape_metric: 0.000000
Epoch 12/20
5947/5947 [==============================] - 5725s 963ms/step - loss: 0.0228 - weighted_mean_squared_error: 0.0228 - weighted_shape_metric: 0.0000e+00 - val_loss: 0.0232 - val_weighted_mean_squared_error: 0.0232 - val_weighted_shape_metric: 0.0000e+00

Epoch 00012: val_loss did not improve from 0.02279
2019-07-11_10:59:07 - Epoch: 12, val_loss: 0.023245, val_weighted_mean_squared_error: 0.023245, val_weighted_shape_metric: 0.000000, loss: 0.022805, weighted_me

In [9]:
import numpy as np
best_epoch,best_val,nepochs=np.loadtxt(dnn2.network_name+".hdf5.best_params")
print(int(best_epoch),best_val,int(nepochs))

17 0.11963585315346718 20


In [9]:
del dnn2

# 3) Alexis net : Restart with gamma=0.1

In [17]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles.hdf5",
             shape_constraint=True,gamma=0.1) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

Loading model...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles.hdf5
Renaming as...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles
Tensor("loss_4/conv2d_24_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window_4:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm_4:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss_4/conv2d_24_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
[<tf.Tensor 'input_3_3:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'window_4:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'norm_4:0' shape=(?, 6, 1, 1) dtype=float32>]
['input_3', 'window', 'norm']


In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.1_restart_newtar_gfiles.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.1_restart_newtar_gfiles.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/10
4949/5947 [=======================>......] - ETA: 16:02 - loss: 0.3430 - weighted_mean_squared_error: 0.0443 - weighted_shape_metric: 0.2987

In [ ]:
del dnn2

# 4) Alexis net : Restart with gamma=0.01

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles.hdf5",
             shape_constraint=True,gamma=0.01) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.01_restart_newtar_gfiles.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

In [ ]:
del dnn2

# 5) Alexis net : Restart with gamma=0.001

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles.hdf5",
             shape_constraint=True,gamma=0.001) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.001_restart_newtar_gfiles.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

# 6) Alexis net : restart with gamma =0.0001

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar_gfiles.hdf5",
             shape_constraint=True,gamma=0.0001) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.0001_restart_newtar_gfiles.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

# Appendix for GPU options

In [ ]:
## extra imports to set GPU options
#import tensorflow as tf
#from keras import backend as k

###################################
# TensorFlow wizardry
#config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
#config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.5 

# Create a session with the above options specified.
#k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Same with multiple GPUs, Tensorflow will pre-allocate the whole memory of all the GPUs.
Use the following to prevent it (only when your station has several GPUs like the SAPPCN63):

In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #"0" for the 1st GPU or "1" to use the 2nd